In [1]:
import requests 

In [2]:
import docx
import spacy
import torch
import numpy as np
from transformers import AutoTokenizer, AutoModel
from sklearn.metrics.pairwise import cosine_similarity
# Load spaCy Russian tokenizer
nlp = spacy.blank("ru")

# Load a pre-trained model tokenizer and model from Hugging Face transformers
model_name = "bert-base-multilingual-cased"  # Pre-trained multilingual BERT model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

def read_docx(file_path):
    # Open the Word document
    doc = docx.Document(file_path)
    
    # Extract text content from the document
    full_text = []
    for para in doc.paragraphs:
        full_text.append(para.text)
    
    # Join all paragraphs into a single string
    document_text = "\n".join(full_text)
    return document_text

def chunk_and_encode_document(document_text, max_chunk_len=600):
    # Split the document into chunks
    doc = nlp(document_text)
    chunks = []
    current_chunk = []

    for token in doc:
        if len(current_chunk) + len(token.text) > max_chunk_len:
            chunks.append(" ".join(current_chunk))
            current_chunk = []

        current_chunk.append(token.text)

    if current_chunk:
        chunks.append(" ".join(current_chunk))

    # Encode each chunk into vectors
    encoded_chunks = []

    for chunk in chunks:
        # Tokenize the chunk
        inputs = tokenizer(chunk, return_tensors="pt", padding=True, truncation=True)

        # Pass the input through the model and get the hidden states
        with torch.no_grad():
            outputs = model(**inputs)

        # Extract the embeddings (CLS token embedding)
        embeddings = outputs.last_hidden_state[:, 0, :]

        # Normalize the embeddings
        normalized_embeddings = torch.nn.functional.normalize(embeddings, p=2, dim=1)

        # Append the normalized embedding to the list
        encoded_chunks.append(normalized_embeddings)

    return chunks, encoded_chunks


file_path = "main_doc.docx"
document_text = read_docx(file_path)
chunks, encoded_chunks = chunk_and_encode_document(document_text)


In [3]:
def encode_query(query_text):
    # Encode the query text into a vector
    inputs = tokenizer(query_text, return_tensors="pt", padding=True, truncation=True)

    with torch.no_grad():
        outputs = model(**inputs)

    # Extract the embeddings (CLS token embedding)
    query_embedding = outputs.last_hidden_state[:, 0, :]

    # Normalize the embedding
    normalized_query_embedding = torch.nn.functional.normalize(query_embedding, p=2, dim=1)

    return normalized_query_embedding



def vector_search(query_embedding, encoded_chunks, chunks):
    # Compute cosine similarity between query embedding and all chunk embeddings
    similarities = []

    for emb in encoded_chunks:
        sim = cosine_similarity(query_embedding.numpy().reshape(1, -1), emb.numpy().reshape(1, -1))
        similarities.append(sim.item())

    # Find indices of the top 3 most similar chunks
    closest_indices = np.argsort(similarities)[::-1][:3]
    closest_chunks = [chunks[idx] for idx in closest_indices]

    return closest_chunks



query = 'За что отвечает академический руководитель?'
query_embedding = encode_query(query)
closest_chunks = vector_search(query_embedding, encoded_chunks, chunks)
i=1
print("Closest chunks to the query:")
for idx, chunk in enumerate(closest_chunks, start=1):
    print('chunk ',i)
    print()
    i+=1
    print(f"{idx}. {chunk}")
    print()

Closest chunks to the query:
chunk  1

1. третьем курсах образовательной программы бакалавриата по итогам учебного года включает две даты пересдачи каждого экзамена вторая   пересдача принимается комиссией в соответствии с правилами проведения последней пересдачи . Дополнительные пересдачи не проводятся независимо от причины пропуска . 
 4.3 . На образовательной программе магистратуры МИЭФ периоды пересдач по итогам 1 - 2 модулей не могут завершаться позднее 15 февраля и по итогам 3 - 4 модулей позднее 15 октября . 
 Расписание пересдач включает не более двух дат пересдачи экзамена ( вторая пересдача принимается комиссией в соответствии с правилами проведения последней пересдачи ) .

chunk  2

2. оценивается по правилам , определенным в ПУД . 
 Блокирующий Элемент контроля – элемент контроля , неудовлетворительная оценка по которому приравнивается к оценке по промежуточной аттестации или по части накопленной оценки до тех пор , пока студент не получит положительную оценку по блокирующе

In [4]:
context = str(closest_chunks)

prompt = {
    "modelUri": "gpt://folder-id/yandexgpt-lite",
    "completionOptions": {
        "stream": False,
        "temperature": 0.6,
        "maxTokens": "2000"
    },
    "messages": [
        {
            "role": "system",
            "text": "ты умный помошник студента, используй документы ниже, чтобы полноценно ответить на поставленный вопрос, если в документах этой информации нет, то так и скажи\
            вот ДОКУМЕНТЫ: " + context
        },
        {
            "role": "user",
            "text": query
        }
    ]
}


url = "https://llm.api.cloud.yandex.net/foundationModels/v1/completion"
headers = {
    "Content-Type": "application/json",
    "Authorization": "Api-Key your key"
}

response = requests.post(url, headers=headers, json=prompt)
result = response.text
response_data = response.json()
assistant_text = response_data['result']['alternatives'][0]['message']['text']
print(assistant_text)

Согласно приведённым документам, академический руководитель **определяет условия** прохождения промежуточной аттестации и текущего контроля **для студентов с ограниченными возможностями здоровья и инвалидностью** с учётом индивидуальных особенностей студента и, в частности, обеспечивает использование доступных средств обучения. Также он **уведомляет менеджера программы** о необходимости индивидуальной организации контроля для таких студентов.

Таким образом, академический руководитель может отвечать за определение конкретных мер по обеспечению доступности промежуточной аттестации и текущего контроля для студентов с инвалидностью и ограниченными возможностями здоровья на образовательной программе.
